# Financial Sentiment Analysis (Single)

In this program, I run a sentiment analysis of a single company based on financial news articles.

The company that I am targeting is Nvidia [NVDA]

## Fetching News Articles

The first step is to fetch the news articles.  

I am using `NewsAPI` to get articles quickly and easily. Then, I use `pandas` to put the articles into a dataframe, where I can collect and read the data easier.  

**Filtering articles:**  
Filter articles that only exist  
- `NewsAPI` sometimes fetches articles that were removed  

**Extracting the data:**  
Extract only the necessary data from the articles
- Title
- Description
- Content

All others can be discarded.  

Both of these steps are part of the cleaning data step that is next in text preprocessing.

In [459]:
import os
from dotenv import load_dotenv

In [460]:
# get path to the environment file
env_path = '../config/.env'
load_dotenv(env_path)

True

In [461]:
# import newsapi package
from newsapi import NewsApiClient

In [462]:
# init newsapi
newsapi = NewsApiClient(api_key=os.getenv('NEWS_API_KEY'))

In [463]:
# fetch all articles that mention Nvidia
all_articles = newsapi.get_everything(q='Nvidia',
                                      language='en')

In [464]:
import pandas as pd
pd.__version__

'2.2.3'

In [465]:
# place all_articles into a dataframe
all_articles_df = pd.DataFrame(all_articles['articles'])
all_articles_df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Yahoo Entertainment'}",Lawrence Bonk,DOJ subpoenas NVIDIA as part of antitrust prob...,The DOJ has sent subpoenas to NVIDIA and other...,https://consent.yahoo.com/v2/collectConsent?se...,None,2024-09-04T15:34:35Z,"If you click 'Accept all', we and our partners..."
1,"{'id': None, 'name': 'Gizmodo.com'}",Kyle Barr,The Leaked Nvidia RTX 5090 Has So Many Cores I...,Get ready to watch the lights on your block di...,https://gizmodo.com/the-leaked-nvidia-rtx-5090...,https://gizmodo.com/app/uploads/2024/09/Nvidia...,2024-09-27T13:35:22Z,The GeForce RTX 4090 is already so big that an...
2,"{'id': None, 'name': '[Removed]'}",None,[Removed],[Removed],https://removed.com,None,2024-08-29T23:30:37Z,[Removed]
3,"{'id': 'business-insider', 'name': 'Business I...",Emma Cosgrove,Nvidia might actually lose in this key part of...,"As AI matures, Nvidia, Groq, and Cerebras focu...",https://www.businessinsider.com/nvidia-may-los...,https://i.insider.com/66d0c408392a3bda9f2349e3...,2024-09-01T13:00:02Z,Justin Sullivan/Getty\r\n<ul><li>Inference mad...
4,"{'id': 'business-insider', 'name': 'Business I...",Eugene Kim,This chart shows one potential advantage AWS's...,"AI chip investments by Amazon, Google, and Mic...",https://www.businessinsider.com/aws-ai-chips-w...,https://i.insider.com/6622c44b23b29110d3011ce1...,2024-09-26T09:00:02Z,Noah Berger/Getty Images\r\n<ul><li>Big tech c...
...,...,...,...,...,...,...,...,...
95,"{'id': None, 'name': 'Hetzner.com'}",None,Hetzner introduces GPU server for AI training,"Find your reliable dream server, optimized for...",https://www.hetzner.com/dedicated-rootserver/g...,None,2024-09-25T11:35:44Z,Discover the next level of performance with th...
96,"{'id': None, 'name': 'Windows Central'}",samuelwtolbert@gmail.com (Samuel Tolbert),"NVIDIA RTX 5080 and 5090 specs seemingly leak,...",The specs for the unrevealed RTX 5080 and 5090...,https://www.windowscentral.com/gaming/pc-gamin...,https://cdn.mos.cms.futurecdn.net/M7rUR7BmMXdM...,2024-09-26T22:22:38Z,What you need to know\r\n<ul><li>The specs for...
97,"{'id': None, 'name': 'Windows Central'}",samuelwtolbert@gmail.com (Samuel Tolbert),MSI teases a STALKER 2-themed NVIDIA RTX graph...,MSI Gaming is teasing a STALKER-2 themed graph...,https://www.windowscentral.com/gaming/pc-gamin...,https://cdn.mos.cms.futurecdn.net/BLkFsTGe9K9C...,2024-09-26T19:25:46Z,What you need to know\r\n<ul><li>MSI is teasin...
98,"{'id': None, 'name': 'XDA Developers'}",Karthik Iyer,How to fix Nvidia GeForce Experience 'driver d...,Let's look at a few troubleshooting methods to...,https://www.xda-developers.com/how-to-fix-nvid...,https://static1.xdaimages.com/wordpress/wp-con...,2024-09-05T10:00:13Z,"If you have a gaming PC, updating drivers is i..."


In [466]:
# filter articles function
# only filters valid articles
# valid meaning: article exists and description of article exists
def filter_removed_articles(articles):
    return [article for article in articles if article.get('title') != '[Removed]']

In [467]:
# filter the all_articles
valid_articles = filter_removed_articles(all_articles['articles'])

In [468]:
valid_articles_df = pd.DataFrame(valid_articles)
valid_articles_df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Yahoo Entertainment'}",Lawrence Bonk,DOJ subpoenas NVIDIA as part of antitrust prob...,The DOJ has sent subpoenas to NVIDIA and other...,https://consent.yahoo.com/v2/collectConsent?se...,None,2024-09-04T15:34:35Z,"If you click 'Accept all', we and our partners..."
1,"{'id': None, 'name': 'Gizmodo.com'}",Kyle Barr,The Leaked Nvidia RTX 5090 Has So Many Cores I...,Get ready to watch the lights on your block di...,https://gizmodo.com/the-leaked-nvidia-rtx-5090...,https://gizmodo.com/app/uploads/2024/09/Nvidia...,2024-09-27T13:35:22Z,The GeForce RTX 4090 is already so big that an...
2,"{'id': 'business-insider', 'name': 'Business I...",Emma Cosgrove,Nvidia might actually lose in this key part of...,"As AI matures, Nvidia, Groq, and Cerebras focu...",https://www.businessinsider.com/nvidia-may-los...,https://i.insider.com/66d0c408392a3bda9f2349e3...,2024-09-01T13:00:02Z,Justin Sullivan/Getty\r\n<ul><li>Inference mad...
3,"{'id': 'business-insider', 'name': 'Business I...",Eugene Kim,This chart shows one potential advantage AWS's...,"AI chip investments by Amazon, Google, and Mic...",https://www.businessinsider.com/aws-ai-chips-w...,https://i.insider.com/6622c44b23b29110d3011ce1...,2024-09-26T09:00:02Z,Noah Berger/Getty Images\r\n<ul><li>Big tech c...
4,"{'id': 'business-insider', 'name': 'Business I...",Emma Cosgrove,Nvidia CEO Jensen Huang says the payback on AI...,Nvidia CEO Jensen Huang promised immediate ret...,https://www.businessinsider.com/nvidia-swift-r...,https://i.insider.com/66cfaf8a43b5e59d16b64ee9...,2024-08-29T00:56:26Z,Jensen Huang said demand will continue to be s...
...,...,...,...,...,...,...,...,...
92,"{'id': None, 'name': 'Hetzner.com'}",None,Hetzner introduces GPU server for AI training,"Find your reliable dream server, optimized for...",https://www.hetzner.com/dedicated-rootserver/g...,None,2024-09-25T11:35:44Z,Discover the next level of performance with th...
93,"{'id': None, 'name': 'Windows Central'}",samuelwtolbert@gmail.com (Samuel Tolbert),"NVIDIA RTX 5080 and 5090 specs seemingly leak,...",The specs for the unrevealed RTX 5080 and 5090...,https://www.windowscentral.com/gaming/pc-gamin...,https://cdn.mos.cms.futurecdn.net/M7rUR7BmMXdM...,2024-09-26T22:22:38Z,What you need to know\r\n<ul><li>The specs for...
94,"{'id': None, 'name': 'Windows Central'}",samuelwtolbert@gmail.com (Samuel Tolbert),MSI teases a STALKER 2-themed NVIDIA RTX graph...,MSI Gaming is teasing a STALKER-2 themed graph...,https://www.windowscentral.com/gaming/pc-gamin...,https://cdn.mos.cms.futurecdn.net/BLkFsTGe9K9C...,2024-09-26T19:25:46Z,What you need to know\r\n<ul><li>MSI is teasin...
95,"{'id': None, 'name': 'XDA Developers'}",Karthik Iyer,How to fix Nvidia GeForce Experience 'driver d...,Let's look at a few troubleshooting methods to...,https://www.xda-developers.com/how-to-fix-nvid...,https://static1.xdaimages.com/wordpress/wp-con...,2024-09-05T10:00:13Z,"If you have a gaming PC, updating drivers is i..."


In [469]:
# extract article essentials function
# extract only the title, description, and content from the articles
def extract_article_essentials(articles):
    return [{'title': article['title'], 'content': article['content']} for article in articles]

In [470]:
extracted_articles = extract_article_essentials(valid_articles)

In [471]:
extracted_articles_df = pd.DataFrame(extracted_articles)
extracted_articles_df

,title,content
0,DOJ subpoenas NVIDIA as part of antitrust prob...,"If you click 'Accept all', we and our partners..."
1,The Leaked Nvidia RTX 5090 Has So Many Cores I...,The GeForce RTX 4090 is already so big that an...
2,Nvidia might actually lose in this key part of...,Justin Sullivan/Getty\r\n<ul><li>Inference mad...
3,This chart shows one potential advantage AWS's...,Noah Berger/Getty Images\r\n<ul><li>Big tech c...
4,Nvidia CEO Jensen Huang says the payback on AI...,Jensen Huang said demand will continue to be s...
...,...,...
92,Hetzner introduces GPU server for AI training,Discover the next level of performance with th...
93,"NVIDIA RTX 5080 and 5090 specs seemingly leak,...",What you need to know\r\n<ul><li>The specs for...
94,MSI teases a STALKER 2-themed NVIDIA RTX graph...,What you need to know\r\n<ul><li>MSI is teasin...
95,How to fix Nvidia GeForce Experience 'driver d...,"If you have a gaming PC, updating drivers is i..."


## Preprocess Text
***This is a crucial***  
Proprocessing helps clean and normalize the text data making it more suitable for analysis.  

After getting the articles, I can now preprocess the text in the articles.    


### Data Cleaning 
**Identify and remove noise:**  
We want to first remove all noise from the data.  
- Punction
- Extra whitespace

**Text normalization:**  
- Stopwords
    - Remove common/irrelevent words that are unlikely to convey much sentiment.  
- Capital letters
    - All letters should be the same case so all words are treated the same in the tokenization process.  

**Data masking:**  
Data masking is not needed in this context.  

Clean text should result.

In [472]:
# import re package (regular expressions)
import re

In [473]:
import string

In [474]:
# import nltk packages (Natural Language Toolkit)
import nltk
from nltk.corpus import stopwords

In [475]:
# download nltk data packages
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/justinhoang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [476]:
# get all stopwords from data package
stop_words = set(stopwords.words("english"))
print(stop_words)

{'few', 'wouldn', 'being', 'itself', 'we', "hadn't", 'at', 'those', 'through', 'on', 'don', 'yourselves', "needn't", 'have', 'do', 'doing', 'or', "aren't", 'so', 'here', 'wasn', 'o', 'yours', 'what', 'own', "you'll", "doesn't", "shan't", 'such', 'hasn', 'after', 'needn', 'she', 'but', "you've", 'only', 'himself', 'these', 'above', "mustn't", "won't", 'then', 'some', 'how', 'them', 'below', 'both', 'will', 'as', 'its', 'where', 'if', 'myself', 'no', "wasn't", 'ma', 'shouldn', 'has', 'hers', 'aren', 'under', 'he', 'other', 'was', 'there', 'to', 'most', 'up', 'by', 'is', 'ain', "shouldn't", 'why', 'which', 'him', 'me', 'be', 're', 'ourselves', 'mustn', 'it', 'in', 'haven', 'about', "it's", 's', 'should', 'a', 'any', 'until', 'our', 'am', 'off', 'again', "haven't", 't', 'been', "isn't", 'whom', 'weren', 'of', 'who', 'doesn', "you're", 'same', "should've", 'your', 'down', 'd', 'and', 'with', 'just', 'ours', 'yourself', 'm', 'now', 'having', 'couldn', "hasn't", 'herself', "she's", "wouldn't"

In [477]:
# clean text function
# cleans the data (text)
def clean_text(text):
    # remove extra whitespace
    cleaned_text = re.sub(r'\s+', ' ', text).strip()
    # remove HTML tags
    cleaned_text = re.sub(r'<[^>]+>', '', cleaned_text)
    # remove patterns
    cleaned_text = re.sub(r'/\w+\s+\w+', '', cleaned_text)
    # remove URLs
    cleaned_text = re.sub(r'http\S+|www\S+|https\S+', '', cleaned_text, flags=re.MULTILINE)
    # remove punctuation
    cleaned_text = ''.join([char for char in text if char not in string.punctuation])
    
    # lower case all text
    cleaned_text = text.lower()
    # remove stop words from text
    cleaned_text = ' '.join([word for word in cleaned_text.split() if word not in stop_words])
    
    return cleaned_text

In [478]:
columns_to_process = ['title', 'content']

In [479]:
cleaned_articles_df = extracted_articles_df.copy()

In [480]:
for column in columns_to_process:
    cleaned_articles_df[column] = extracted_articles_df[column].apply(clean_text)

In [481]:
cleaned_articles_df.columns = columns_to_process

In [482]:
cleaned_articles_df

,title,content
0,doj subpoenas nvidia part antitrust probe rega...,"click 'accept all', partners, including 239 pa..."
1,leaked nvidia rtx 5090 many cores actually scares,geforce rtx 4090 already big pc builder worth ...
2,nvidia might actually lose key part ai chip bu...,justin sullivan/getty <ul><li>inference made 4...
3,chart shows one potential advantage aws's ai c...,noah berger/getty images <ul><li>big tech clou...
4,nvidia ceo jensen huang says payback ai spendi...,jensen huang said demand continue stronger sup...
...,...,...
92,hetzner introduces gpu server ai training,discover next level performance new gex130 ded...
93,"nvidia rtx 5080 5090 specs seemingly leak, rev...",need know <ul><li>the specs nvidia's upcoming ...
94,msi teases stalker 2-themed nvidia rtx graphic...,need know <ul><li>msi teasing new graphics car...
95,fix nvidia geforce experience 'driver download...,"gaming pc, updating drivers important maintain..."


### Linguistic Proccesing

I break up each of these processes into their own methods to ensure they are being processed correctly.  
Then I apply them all at once.

**Tokenization**  
Breaking down text into individual words or tokens. Helps separate individual words from the raw text, which makes it easier to analyze and understand.

**POS Tagging**  
Part-of-Speech Tagging

**Lemmatization**  
A technique to reduce words to their root forms based on their part of speech and the meaning of a word based on its context.  
An algorithmic process of identifying an inflected word's "lemma" based on its intended meaning. The algorithm we are utilizing is apart of the `NLTK` library.

**Named-Entity Recoginition**


#### Tokenization

In [484]:
# import nltk tokenization module
from nltk.tokenize import word_tokenize


In [485]:
# import nltk punkt word tokenization data package
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/justinhoang/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [486]:
def tokenize(text):
    return word_tokenize(text)

In [487]:
tokenized_articles_df = cleaned_articles_df.copy()

In [488]:
for column in columns_to_process:
    for index, row in cleaned_articles_df.iterrows():
        try:
            tokenized_articles_df.at[index, column] = tokenize(row[column])
        except Exception as e:
            print(e)

In [489]:
tokenized_articles_df

,title,content
0,"[doj, subpoenas, nvidia, part, antitrust, prob...","[click, 'accept, all, ', ,, partners, ,, inclu..."
1,"[leaked, nvidia, rtx, 5090, many, cores, actua...","[geforce, rtx, 4090, already, big, pc, builder..."
2,"[nvidia, might, actually, lose, key, part, ai,...","[justin, sullivan/getty, <, ul, >, <, li, >, i..."
3,"[chart, shows, one, potential, advantage, aws,...","[noah, berger/getty, images, <, ul, >, <, li, ..."
4,"[nvidia, ceo, jensen, huang, says, payback, ai...","[jensen, huang, said, demand, continue, strong..."
...,...,...
92,"[hetzner, introduces, gpu, server, ai, training]","[discover, next, level, performance, new, gex1..."
93,"[nvidia, rtx, 5080, 5090, specs, seemingly, le...","[need, know, <, ul, >, <, li, >, the, specs, n..."
94,"[msi, teases, stalker, 2-themed, nvidia, rtx, ...","[need, know, <, ul, >, <, li, >, msi, teasing,..."
95,"[fix, nvidia, geforce, experience, 'driver, do...","[gaming, pc, ,, updating, drivers, important, ..."


#### POS Tagging

In [490]:
# import nltk pos tagging module
from nltk import pos_tag

In [491]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/justinhoang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [492]:
def pos_tag_text(tokens):
    return pos_tag(tokens)

In [493]:
pos_tagged_articles_df = tokenized_articles_df.copy()

In [494]:
for column in columns_to_process:
    for index, row in tokenized_articles_df.iterrows():
        try:
            pos_tagged_articles_df.at[index, column] = pos_tag_text(row[column])
        except Exception as e:
            print(e)

In [495]:
pos_tagged_articles_df

,title,content
0,"[(doj, NN), (subpoenas, NNS), (nvidia, VBP), (...","[(click, NN), ('accept, IN), (all, DT), (', ''..."
1,"[(leaked, VBN), (nvidia, RB), (rtx, JJ), (5090...","[(geforce, NN), (rtx, NN), (4090, CD), (alread..."
2,"[(nvidia, NN), (might, MD), (actually, RB), (l...","[(justin, NN), (sullivan/getty, JJ), (<, NNP),..."
3,"[(chart, NN), (shows, VBZ), (one, CD), (potent...","[(noah, JJ), (berger/getty, NN), (images, NNS)..."
4,"[(nvidia, RB), (ceo, NN), (jensen, NN), (huang...","[(jensen, NN), (huang, NN), (said, VBD), (dema..."
...,...,...
92,"[(hetzner, NN), (introduces, NNS), (gpu, VBP),...","[(discover, RB), (next, JJ), (level, NN), (per..."
93,"[(nvidia, RB), (rtx, NN), (5080, CD), (5090, C...","[(need, NN), (know, VBP), (<, NNP), (ul, JJ), ..."
94,"[(msi, NN), (teases, VBZ), (stalker, JJR), (2-...","[(need, NN), (know, VBP), (<, NNP), (ul, JJ), ..."
95,"[(fix, JJ), (nvidia, JJ), (geforce, NN), (expe...","[(gaming, VBG), (pc, NN), (,, ,), (updating, V..."


#### Lemmatization

In [496]:
# import nltk lemmatization modules
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [497]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/justinhoang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [498]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [499]:
def lemmatize(tagged_tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged_tokens]

In [500]:
lemmatized_articles_df = pos_tagged_articles_df.copy()

In [501]:
for column in columns_to_process:
    for index, row in pos_tagged_articles_df.iterrows():
        try:
            lemmatized_articles_df.at[index, column] = lemmatize(row[column])
        except Exception as e:
            print(e)

In [502]:
lemmatized_articles_df

,title,content
0,"[doj, subpoena, nvidia, part, antitrust, probe...","[click, 'accept, all, ', ,, partner, ,, includ..."
1,"[leak, nvidia, rtx, 5090, many, core, actually...","[geforce, rtx, 4090, already, big, pc, builder..."
2,"[nvidia, might, actually, lose, key, part, ai,...","[justin, sullivan/getty, <, ul, >, <, li, >, i..."
3,"[chart, show, one, potential, advantage, aws, ...","[noah, berger/getty, image, <, ul, >, <, li, >..."
4,"[nvidia, ceo, jensen, huang, say, payback, ai,...","[jensen, huang, say, demand, continue, strong,..."
...,...,...
92,"[hetzner, introduces, gpu, server, ai, train]","[discover, next, level, performance, new, gex1..."
93,"[nvidia, rtx, 5080, 5090, spec, seemingly, lea...","[need, know, <, ul, >, <, li, >, the, specs, n..."
94,"[msi, tease, stalker, 2-themed, nvidia, rtx, g...","[need, know, <, ul, >, <, li, >, msi, tease, n..."
95,"[fix, nvidia, geforce, experience, 'driver, do...","[game, pc, ,, update, driver, important, maint..."


#### Named Entity Recognition

In [503]:
# import nltk named entity modules
from nltk.chunk import ne_chunk

In [511]:
# download nltk ne data packages
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /Users/justinhoang/nltk_data...
[nltk_data]   Package maxent_ne_chunker_tab is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/justinhoang/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [512]:
def ner(pos_tags):
    return ne_chunk(pos_tags)

#### Linguistic Process

In [518]:
def linguistic_process(text):
    tokens = tokenize(text)
    pos_tags = pos_tag_text(tokens)
    lemmatized = lemmatize(pos_tags)
    ner_tree = ner(pos_tags)
    
    processed_text = ' '.join(lemmatized)
    return processed_text, ner_tree

### Preprocess the Text

In [520]:
def preprocess_text(text):
    cleaned = clean_text(text)
    processed, ner = linguistic_process(cleaned)
    return processed, ner

In [521]:
processed_articles_df = pd.DataFrame()

In [522]:
for column in columns_to_process:
    processed_articles_df[f'processed_{column}'], processed_articles_df[f'ner_{column}'] = zip(*extracted_articles_df[column].apply(preprocess_text))

In [523]:
processed_articles_df

,processed_title,ner_title,processed_content,ner_content
0,doj subpoena nvidia part antitrust probe regar...,"[(doj, NN), (subpoenas, NNS), (nvidia, VBP), (...","click 'accept all ' , partner , include 239 pa...","[(click, NN), ('accept, IN), (all, DT), (', ''..."
1,leak nvidia rtx 5090 many core actually scare,"[(leaked, VBN), (nvidia, RB), (rtx, JJ), (5090...",geforce rtx 4090 already big pc builder worth ...,"[(geforce, NN), (rtx, NN), (4090, CD), (alread..."
2,nvidia might actually lose key part ai chip bu...,"[(nvidia, NN), (might, MD), (actually, RB), (l...",justin sullivan/getty < ul > < li > inference ...,"[(justin, NN), (sullivan/getty, JJ), (<, NNP),..."
3,chart show one potential advantage aws 's ai c...,"[(chart, NN), (shows, VBZ), (one, CD), (potent...",noah berger/getty image < ul > < li > big tech...,"[(noah, JJ), (berger/getty, NN), (images, NNS)..."
4,nvidia ceo jensen huang say payback ai spendin...,"[(nvidia, RB), (ceo, NN), (jensen, NN), (huang...",jensen huang say demand continue strong supply...,"[(jensen, NN), (huang, NN), (said, VBD), (dema..."
...,...,...,...,...
92,hetzner introduces gpu server ai train,"[(hetzner, NN), (introduces, NNS), (gpu, VBP),...",discover next level performance new gex130 ded...,"[(discover, RB), (next, JJ), (level, NN), (per..."
93,"nvidia rtx 5080 5090 spec seemingly leak , rev...","[(nvidia, RB), (rtx, NN), (5080, CD), (5090, C...",need know < ul > < li > the specs nvidia 's up...,"[(need, NN), (know, VBP), (<, NNP), (ul, JJ), ..."
94,msi tease stalker 2-themed nvidia rtx graphic ...,"[(msi, NN), (teases, VBZ), (stalker, JJR), (2-...",need know < ul > < li > msi tease new graphic ...,"[(need, NN), (know, VBP), (<, NNP), (ul, JJ), ..."
95,fix nvidia geforce experience 'driver download...,"[(fix, JJ), (nvidia, JJ), (geforce, NN), (expe...","game pc , update driver important maintain pc ...","[(gaming, VBG), (pc, NN), (,, ,), (updating, V..."
